In [1]:
%pip install --upgrade git+https://github.com/keras-team/keras-cv -q --break-system-packages
# %pip install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 --break-system-packages

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization
import cv2
import random

import numpy as np

/home/gabriela/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-02 17:53:56.284337: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 17:53:56.478239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733172836.552644     927 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733172836.573067     927 cuda_blas.cc:1418] Unable t

In [3]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0
IMAGE_WIDTH = 3000
IMAGE_HEIGHT = 4000

TF_ENABLE_ONEDNN_OPTS=0
TF_DISABLE_MKL=1

In [4]:
class_ids = [
    "neutrofilo",
    "linfocito",
    "monocito",
    "bastonete",
    "metamielocito",
    "eosinofilo",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

# Path to images and annotations
path_images = "/home/gabriela/projetos/yolov8keras/Bach1"
path_annot = "/home/gabriela/projetos/yolov8keras/Bach1/annotations"

path_test_img = "/home/gabriela/projetos/yolov8keras/test"
path_test_annot = "/home/gabriela/projetos/yolov8keras/test/annotations"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(path_annot, file_name)
        for file_name in os.listdir(path_annot)
        if file_name.endswith(".xml")
    ]
)

xml_test_files = sorted(
    [
        os.path.join(path_test_annot, file_name)
        for file_name in os.listdir(path_test_annot)
        if file_name.endswith(".xml")
    ]
)

# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(".jpg")
    ]
)

jpg_test_files = sorted(
    [
        os.path.join(path_test_img, file_name)
        for file_name in os.listdir(path_test_img)
        if file_name.endswith(".jpg")
    ]
)

In [5]:
class_mapping[0]

'neutrofilo'

In [6]:
#read the objects in the annotation xml file
def parse_annotation(xml_file, path_img):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(path_img, image_name)

    boxes = []
    classes = []
    for obj in root.find("objects"):
        cls = obj.tag
        # print(cls)
        classes.append(cls)

        if(obj.find("bbox") != None):
            bbox = obj.find("bbox")
            min_x, min_y, width, height = normalize_bounding_box(float(bbox.find("x").text), float(bbox.find("y").text), float(bbox.find("width").text), float(bbox.find("height").text))
            boxes.append([min_x, min_y, width, height])
        else:
            xCoords = []
            yCoords = []
            for coord in obj:
                if (coord.tag.find("x") != -1):
                    xCoords.append(float(coord.text))
                elif (coord.tag.find("y") != -1):
                    yCoords.append(float(coord.text))

            xmin, ymin, width, height = calculate_bounding_box_normalized(xCoords, yCoords)
            boxes.append([xmin, ymin, width, height])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids

#calculate bounding boxes with points provided
def calculate_bounding_box_normalized(xCoords, yCoords):
    points = np.array([xCoords, yCoords])

    min_x = float(np.min(points[0, :]))
    min_x = (min_x/IMAGE_WIDTH)*640
    min_y = float(np.min(points[1, :]))
    # min_y = ((IMAGE_HEIGHT - min_y)/IMAGE_HEIGHT)*640
    min_y = (min_y/IMAGE_HEIGHT)*640

    max_x = float(np.max(points[0, :]))
    max_x = (max_x/IMAGE_WIDTH)*640
    max_y = float(np.max(points[1, :]))
    # max_y = ((IMAGE_HEIGHT - max_y)/IMAGE_HEIGHT)*640
    max_y = (max_y/IMAGE_HEIGHT)*640

    width =  max_x - min_x
    height = max_y - min_y

    return (min_x, min_y, width, height)

def normalize_bounding_box(xmin, ymin, width, height):

    min_x = (xmin/IMAGE_WIDTH)*640
    # min_y = ((IMAGE_HEIGHT - ymin)/IMAGE_HEIGHT)*640
    min_y = (ymin/IMAGE_HEIGHT)*640
    width = (width/IMAGE_WIDTH)*640
    height = (height/IMAGE_HEIGHT)*640

    return (min_x, min_y, width, height)

In [7]:
image_paths = []
bbox = []
classes = []

for xml_file in tqdm(xml_files):
    image_path, boxes, class_ids = parse_annotation(xml_file, path_images)
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)

100%|██████████| 245/245 [00:00<00:00, 2877.73it/s]


In [8]:
image_test_paths = []
bbox_test = []
classes_test = []

for xml_file in tqdm(xml_test_files):
    image_path, boxes, class_ids = parse_annotation(xml_file, path_test_img)
    image_test_paths.append(image_path)
    bbox_test.append(boxes)
    classes_test.append(class_ids)

100%|██████████| 100/100 [00:00<00:00, 1943.23it/s]


In [9]:
def load_image(path):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (640, 640), interpolation=cv2.INTER_CUBIC)
    return image

def load_all_images(paths):
    images = []
    for path in paths:
        image = load_image(path)
        images.append(image)
        # print(image.shape)
    return images


images = load_all_images(image_paths)

test_images = load_all_images(image_test_paths)

In [25]:
images[100].shape

TensorShape([640, 640, 3])

In [ ]:
bbox[100][0]

[211.64500395373847, 260.3884766351209, 27.2868875067318, 20.976794770800076]

In [12]:
classes[100]

[3]

In [13]:
def drawBoundingBoxes(imageData, imageOutputPath, bboxes, labels, color):
    """Draw bounding boxes on an image.
    imageData: image data in numpy array format
    imageOutputPath: output image file path
    inferenceResults: inference results array off object (l,t,w,h)
    colorMap: Bounding box color candidates, list of RGB tuples.
    """
    i = 0
    for box in bboxes:
        left = int(box[0])
        bottom = int(box[1])
        right = int(box[0]) + int(box[2])
        top = int(box[1]) + int(box[3])
        label = class_mapping[int(labels[i])]
        i+=1
        imgHeight, imgWidth, _ = imageData.shape
        thick = int((imgHeight + imgWidth) // 900)
        print (left, bottom, imgHeight, imgWidth)
        print (label)
        cv2.rectangle(imageData,(left, top), (right, bottom), color, thick)
        cv2.putText(imageData, label, (left, top - 20), 0, 1e-3 * imgHeight, color, thick//3)
    cv2.imwrite(imageOutputPath, imageData)

In [14]:
# print(len(images))
index = 100
drawBoundingBoxes(images[index], '/home/gabriela/projetos/yolov8keras/output/example_Bach1.jpg', bbox[index], classes[index], (0, 0, 255))

211 260 640 640
bastonete


In [15]:
images = tf.convert_to_tensor(images, dtype=tf.float32)
# print(images.shape)

I0000 00:00:1733172864.693854     927 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [24]:
ragged_bboxes = tf.ragged.constant(bbox, dtype=tf.float32)
ragged_classes = tf.ragged.constant(classes, dtype=tf.int64)

labels = {
    "boxes": ragged_bboxes.to_tensor(),
    "classes": ragged_classes.to_tensor(),
}

# print(labels["boxes"].type())
print(len(labels["classes"]))

245


In [17]:
# model = keras_cv.models.YOLOV8Detector(
#     num_classes=len(class_mapping),
#     bounding_box_format="xywh",
#     backbone=keras_cv.models.YOLOV8Backbone.from_preset(
#         "yolo_v8_s_backbone_coco"
#     ),
#     fpn_depth=2
# )

backbone=keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_xs_backbone_coco")

model = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xywh",
    backbone=backbone,
    fpn_depth=2,
)

In [18]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

model.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)

# model.compile(
#     classification_loss='binary_crossentropy',
#     box_loss='ciou',
#     optimizer=tf.optimizers.SGD(global_clipnorm=10.0),
#     jit_compile=False,
# )

In [19]:
model.fit(images, labels, batch_size=4, epochs=10)

Epoch 1/10


/home/gabriela/.local/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
I0000 00:00:1733172987.926774    6820 service.cc:148] XLA service 0x7f9e48002510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733172987.927278    6820 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-12-02 17:56:28.795903: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-02 17:56:30.296614: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at bcast_ops.cc:52 : INVALID_ARGUMENT: Input 0 to node `yolov8_label_encoder_1/BroadcastArgs` with op BroadcastArgs must be a compile-time constant.

XLA compilation requires 

InvalidArgumentError: Graph execution error:

Detected at node yolov8_label_encoder_1/BroadcastArgs defined at (most recent call last):
<stack traces unavailable>
Input 0 to node `yolov8_label_encoder_1/BroadcastArgs` with op BroadcastArgs must be a compile-time constant.

XLA compilation requires that operator arguments that represent shapes or dimensions be evaluated to concrete values at compile time. This error means that a shape or dimension argument could not be evaluated at compile time, usually because the value of the argument depends on a parameter to the computation, on a variable, or on a stateful operation such as a random number generator.

Stack trace for op definition: 
File "<frozen runpy>", line 198, in _run_module_as_main
File "<frozen runpy>", line 88, in _run_code
File "/home/gabriela/.local/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
File "/home/gabriela/.local/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, 

In [ ]:
y_pred = model.predict(test_images)
print(y_pred)